In [11]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np
import BacktestingStrategies.Optimization as opt

#BTCJPY2時間足
#http://nipper.work/btc/index.php?market=bitFlyer&coin=BTCJPY&periods=7200&after=1640995200&before=1665100800
data = pd.read_csv('btc.csv', names=("Date", "Open", "High", "Low", "Close"))
data["Date"] = pd.to_datetime(data.index)
data.set_index('Date', inplace = True)
data = data.astype("double")

initial = 10_000_000
fname = f"html/plot-"
df_returns = pd.DataFrame()

In [12]:
print(data)

                          Open       High        Low       Close
Date                                                            
2022-01-01 00:00:00  5359000.0  5316222.0  5325242.0  190.414749
2022-01-01 02:00:00  5407484.0  5323749.0  5388884.0  212.975996
2022-01-01 04:00:00  5402039.0  5380820.0  5391012.0   90.025930
2022-01-01 06:00:00  5471828.0  5368104.0  5430000.0  202.119379
2022-01-01 08:00:00  5442174.0  5401395.0  5434087.0   88.946301
...                        ...        ...        ...         ...
2022-10-05 22:00:00  2915654.0  2891547.0  2898364.0  175.347057
2022-10-06 00:00:00  2922090.0  2893374.0  2912383.0   69.862984
2022-10-06 02:00:00  2950000.0  2909969.0  2945261.0  212.950939
2022-10-06 04:00:00  2954618.0  2931490.0  2941585.0  102.436067
2022-10-06 06:00:00  2955660.0  2927715.0  2935521.0  116.951120

[3336 rows x 4 columns]


In [13]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [14]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [15]:
import BacktestingStrategies.Strategy_MacdCross as btst
importlib.reload(btst)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btst.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "MACDThreshold": range(-100, 100, 10),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000
}
execute_optimize(bt, opt_params)


# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
execute_optimize(bt, opt_params)

Start                     2022-01-01 00:00:00
End                       2022-10-06 06:00:00
Duration                    278 days 06:00:00
Exposure Time [%]                   47.092326
Equity Final [$]                  6280563.366
Equity Peak [$]                11738350.52715
Return [%]                         -37.194366
Buy & Hold Return [%]               -38.58085
Return (Ann.) [%]                  -45.583208
Volatility (Ann.) [%]                     inf
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -99.38918
Avg. Drawdown [%]                  -95.156825
Max. Drawdown Duration      237 days 18:00:00
Avg. Drawdown Duration       66 days 02:00:00
# Trades                                   52
Win Rate [%]                        42.307692
Best Trade [%]                      13.015148
Worst Trade [%]                    -11.258518
Avg. Trade [%]                    

c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,94,122,4.804666e+06,4892026.0,1.747201e+05,0.018182,2022-01-08 20:00:00,2022-01-11 04:00:00,2 days 08:00:00
1,2,151,167,5.056408e+06,4963844.0,-1.851279e+05,-0.018306,2022-01-13 14:00:00,2022-01-14 22:00:00,1 days 08:00:00
2,1,177,208,5.004139e+06,4869500.0,-1.346395e+05,-0.026906,2022-01-15 18:00:00,2022-01-18 08:00:00,2 days 14:00:00
3,2,278,289,4.093121e+06,4185000.0,1.837577e+05,0.022447,2022-01-24 04:00:00,2022-01-25 02:00:00,0 days 22:00:00
4,2,307,373,4.401681e+06,4450000.0,9.663800e+04,0.010977,2022-01-26 14:00:00,2022-02-01 02:00:00,5 days 12:00:00
5,2,403,423,4.250758e+06,4804000.0,1.106485e+06,0.130151,2022-02-03 14:00:00,2022-02-05 06:00:00,1 days 16:00:00
6,2,441,457,4.836215e+06,5084474.0,4.965173e+05,0.051333,2022-02-06 18:00:00,2022-02-08 02:00:00,1 days 08:00:00
7,2,487,495,5.245838e+06,5042224.0,-4.072280e+05,-0.038814,2022-02-10 14:00:00,2022-02-11 06:00:00,0 days 16:00:00
8,2,514,544,4.991574e+06,5052968.0,1.227888e+05,0.012300,2022-02-12 20:00:00,2022-02-15 08:00:00,2 days 12:00:00
9,2,560,579,5.082648e+06,4702699.0,-7.598984e+05,-0.074754,2022-02-16 16:00:00,2022-02-18 06:00:00,1 days 14:00:00


c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501
c:\Python310\

Start                     2022-01-01 00:00:00
End                       2022-10-06 06:00:00
Duration                    278 days 06:00:00
Exposure Time [%]                    0.839329
Equity Final [$]                10036427.0322
Equity Peak [$]                 10036427.0322
Return [%]                            0.36427
Buy & Hold Return [%]               -38.58085
Return (Ann.) [%]                    0.476822
Volatility (Ann.) [%]           365748.840103
Sharpe Ratio                         0.000001
Sortino Ratio                         0.00536
Calmar Ratio                         0.006142
Max. Drawdown [%]                  -77.630054
Avg. Drawdown [%]                  -77.630054
Max. Drawdown Duration        2 days 08:00:00
Avg. Drawdown Duration        2 days 08:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       0.469222
Worst Trade [%]                      0.469222
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,1606,1633,3.881641e+06,3899855.0,36427.0322,0.004692,2022-05-14 20:00:00,2022-05-17 02:00:00,2 days 06:00:00


<Strategy MACDCross(MACDshort=15,MACDlong=20,MACDsignal=10,MACDThreshold=-30)>